# Différentes techniques d'évaluation d'un système de recommandation

La qualité d'un système de recommandation peut être déterminée à l'aide de différentes métriques qui font l'objet de cette présentation.
On en distingue deux principales :
1. L'erreur moyenne absolue et l'erreur moyenne absolue normalisée
2. L'erreur quadratique moyenne

Les sources utilisées et la documentation est généralement en anglais. Pour une meilleure compréhension, voici les équivalents dans la langue de Shakespeare :

Français | Anglais
------------ | -------------
Erreur Moyenne Absolue (EMA) | Mean Absolute Error (MAE)
Erreur Quadratique Moyenne | Root Mean Squared Error (RMSE)

## Les métriques en théorie

Dans un premier temps nous nous attacherons à décrire la théorie mathématique derrière chacune de ces métriques, leurs avantages et inconvénients.

### Mean Absolute Error (MAE)

Il s'agit de la moyenne de la somme des écarts entre les valeurs prédites et les valeurs réelles, en valeur absolue.

$$ MAE = \frac{1}{n} \sum\limits_{j=1}^n \lvert p_{i,j} - r_{i,j} \rvert $$

$n$ est le nombre total de notes données par l'utilisateur $i$. $p_{i,j}$ est la note prédite pour l'utilisateur $i$ et l'item $j$ (exemples : un film, une chanson, un livre ...) et $r_{i,j}$ la note réelle accordée par l'utilisateur. Nous avons étudié le cas où on cherché l'erreur pour un utilisateur mais on pourrait imaginer calculer la MAE sur l'ensemble de la matrice d'interaction ainsi :

$$ MAE = \frac{1}{n} \sum\limits_{i=1}^n \sum\limits_{j=1}^n \lvert p_{i,j} - r_{i,j} \rvert $$

Pour chaque utilisateur (chaque ligne de notre matrice d'interaction), on calcule l'erreur pour chaque chaque item noté. Plus la MAE est faible, meilleure est la prédiction.

Pour plus de détails sur cette métrique :
* [Papier de chercheurs de l'UC Berkeley](http://goldberg.berkeley.edu/pubs/eigentaste.pdf)
* [La référence en matière d'évaluation de système de recommandation](https://grouplens.org/site-content/uploads/evaluating-TOIS-20041.pdf)

### Root Mean Squared Error (RMSE)

C'est la métrique utilisée lors du prix Netflix. La formule est la suivante :

$$ RMSE = \sqrt{ \frac{1}{n} \sum\limits_{i=1}^n \sum\limits_{j=1}^n  (p_{i,j} - r_{i,j})^2 } $$

$n$ est le nombre total de notes pour tous les utilisateurs et items. $p_{i,j}$ est la note prédite pour l'utilisateur $i$ et l'item $j$ et $r_{i,j}$ la note réelle. Comme pour la MAE, plus la RMSE est faible, meilleure est la prédiction. 

### Discussion

Un système de recommandation précis (avec des bonnes métriques) n'est pas forcément le meilleur aux yeux de l'utilisateur. L'utilisateur n'a pas forcément envie d'avoir du contenu basé sur ses anciens favoris et peut préférer de nouvelles choses. C'est la raison pour laquelle d'autres métriques comme la sensibilité ROC peuvent être utilisées.

## Les métriques en pratique

En Python, comment calculer ces métriques ?

In [49]:
import math # to do some basic operations like square root
import numpy as np# linear algebra
from sklearn.metrics import mean_absolute_error, mean_squared_error

### Mean Absolute Error

In [50]:
# By hand
def get_mae(predicted_matrix, real_matrix):
    """function that computes the Mean Absolute Error (MAE) given two matrices, the first one being the 
    predicted matrix, the second one being the matrix that contains the real ratings."""
    absolute_error = 0
    
    # sum absolute errors
    for i in range(np.size(predicted_matrix, 0)): # for each user (row)
        for j in range(np.size(predicted_matrix, 1)): # for each item (column)
            absolute_error += abs(predicted_matrix[i][j] - real_matrix[i][j])

    return absolute_error / np.size(predicted_matrix) # divide by the total number of ratings

In [51]:
# Using scikit-learn module
def get_scikit_mae(predicted_matrix, real_matrix):
    """function that returns the MAE using scikit-learn module"""
    return mean_absolute_error(real_matrix, predicted_matrix)

In [52]:
# generate matrices of random numbers
dim = (3, 5) # 3 users, 5 items
predicted = np.random.randint(1, 6, size=dim)
real = np.random.randint(1, 6, size=dim)
print('Mean Absolute Error with my function : {0:.2f}'.format(get_mae(predicted, real)))
print('Mean Absolute Error with scikit-learn : {0:.2f}'.format(get_scikit_mae(predicted, real)))

Mean Absolute Error with my function : 1.93
Mean Absolute Error with scikit-learn : 1.93


In [53]:
# For big matrices, computation times are essential. So let's compare the two methods
dim = (200, 500)
predicted = np.random.randint(1, 6, size=dim)
real = np.random.randint(1, 6, size=dim) 
%timeit get_mae(predicted, real)
%timeit get_scikit_mae(predicted, real)

70.2 ms ± 485 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
727 µs ± 141 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Root Mean Squared Error

In [54]:
# By hand
def get_rmse(predicted_matrix, real_matrix):
    """function that computes the Root Mean Squared Error (RMSE) given two matrices, the first one being the 
    predicted matrix, the second one being the matrix that contains the real ratings."""
    squared_error = 0
    
    # sum squared errors
    for i in range(np.size(predicted_matrix, 0)): # for each user (row)
        for j in range(np.size(predicted_matrix, 1)): # for each item (column)
            squared_error += (predicted_matrix[i][j] - real_matrix[i][j]) ** 2
    
    return math.sqrt(squared_error / np.size(predicted_matrix))

In [55]:
# Using scikit-learn module
def get_scikit_rmse(predicted_matrix, real_matrix):
    """function that returns the RMSE using scikit-learn module"""
    # scikit-learn returns the mean squared error and we want the ROOT mean squared error, so we need to use math.sqrt function
    return math.sqrt(mean_squared_error(real_matrix, predicted_matrix))

In [56]:
# generate matrices of random numbers
dim = (5, 7) # 5 users, 7 items
predicted = np.random.randint(1, 6, size=dim)
real = np.random.randint(1, 6, size=dim)
print('RMSE with my function : {0:.2f}'.format(get_rmse(predicted, real)))
print('RMSE with scikit-learn : {0:.2f}'.format(get_scikit_rmse(predicted, real)))

RMSE with my function : 2.13
RMSE with scikit-learn : 2.13


In [57]:
# For big matrices, computation times are essential. So let's compare the two methods
dim = (200, 500)
predicted = np.random.randint(1, 6, size=dim)
real = np.random.randint(1, 6, size=dim) 
%timeit get_rmse(predicted, real)
%timeit get_scikit_rmse(predicted, real)

84 ms ± 2.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
581 µs ± 3.98 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
